In [1]:
import numpy as np
import matplotlib.pyplot as plt

4.5 計算例

4.5.1 衝撃波管問題

In [ ]:
def get_U_parameters_1(rho, u, v, w, Bx, By, Bz, e, pT_1, S, SM):
    u1 = SM
    v1 = v - Bx * By * (SM - u) / (rho * (S - u) * (S - SM) - Bx**2)
    w1 = w - Bz * By * (SM - u) / (rho * (S - u) * (S - SM) - Bx**2)
    Bx1 = Bx #1次元だからこれでいいはず
    By1 = By * (rho * (S - u)**2 - Bx**2) / (rho * (S - u) * (S - SM) - Bx**2)
    Bz1 = Bz * (rho * (S - u)**2 - Bx**2) / (rho * (S - u) * (S - SM) - Bx**2)
    e1 = ((S - u) * e - pT_1 * u + pT_1 * SM + Bx * ((u*Bx + v*By + w*Bz) - (u1*Bx1 + v1*By1 + w1*Bz1))) / (S - SM)
    return u1, v1, w1, Bx1, By1, Bz1, e1


def get_U_parameters_2(rho1_L, rho1_R, u1_L, u1_R, v1_L, v1_R, w1_L, w1_R, Bx1_L, Bx1_R, By1_L, By1_R, Bz1_L, Bz1_R, e1_L, e1_R, SM):
    u2 = SM
    Bx2 = Bx1_L #1次元だからこれでいいはず
    v2 = (np.sqrt(rho1_L)*v1_L + np.sqrt(rho1_R)*v1_R + (By1_R - By1_L)*np.sign(Bx2)) / (np.sqrt(rho1_L) + np.sqrt(rho1_R))
    w2 = (np.sqrt(rho1_L)*w1_L + np.sqrt(rho1_R)*w1_R + (Bz1_R - Bz1_L)*np.sign(Bx2)) / (np.sqrt(rho1_L) + np.sqrt(rho1_R))
    By2 = (np.sqrt(rho1_L)*By1_R + np.sqrt(rho1_R)*By1_L + np.sqrt(rho1_L*rho1_R)*(v1_R - v1_L)*np.sign(Bx2)) / (np.sqrt(rho1_L) + np.sqrt(rho1_R))
    Bz2 = (np.sqrt(rho1_L)*Bz1_R + np.sqrt(rho1_R)*Bz1_L + np.sqrt(rho1_L*rho1_R)*(w1_R - w1_L)*np.sign(Bx2)) / (np.sqrt(rho1_L) + np.sqrt(rho1_R))
    e2_L = e1_L - np.sqrt(rho1_L) * ((u1_L*Bx1_L + v1_L*By1_L + w1_L*Bz1_L) - (u2*Bx2 + v2*By2 + w2*Bz2)) * np.sign(Bx2)
    e2_R = e1_R - np.sqrt(rho1_R) * ((u1_R*Bx1_R + v1_R*By1_R + w1_R*Bz1_R) - (u2*Bx2 + v2*By2 + w2*Bz2)) * np.sign(Bx2)

    return u2, v2, w2, Bx2, By2, Bz2, e2_L, e2_R

In [3]:
dx = 0.001
dt = 0.0005
x = np.arange(0.0, 1.0, dx)
nmax = len(x)

gamma = 5.0/3.0

U = np.zeros([x.shape[0], 8]) # rho, rho*u, rho*v, rho*w, Bx, By, Bz, e
# ここでのLとRはメインループ内のL・Rの意味とは違う
# ここのは左半分・右半分の意味
rho_L_init = 1.0
rho_R_init = 0.125
velocity_L_init = np.array([0.0, 0.0, 0.0])
velocity_R_init = np.array([0.0, 0.0, 0.0])
B_L_init = np.array([0.75, 1.0, 0.0])
B_R_init = np.array([0.75, -1.0, 0.0])
p_L_init = 1.0
p_R_init = 0.1
U[x < 0.5, 0] = rho_L_init
U[x >= 0.5, 0] = rho_R_init
U[x < 0.5, 1:4] = rho_L_init * velocity_L_init
U[x >= 0.5, 1:4] = rho_R_init * velocity_R_init
U[x < 0.5, 4:7] = B_L_init
U[x >= 0.5, 4:7] = B_R_init
U[x < 0.5, 7] = p_L_init/(gamma-1) + rho_L_init * np.linalg.norm(velocity_L_init)**2/2 + np.linalg.norm(B_L_init)**2/2
U[x >= 0.5, 7] = p_R_init/(gamma-1) + rho_R_init * np.linalg.norm(velocity_R_init)**2/2 + np.linalg.norm(B_R_init)**2/2

F = np.zeros(U.shape)

In [7]:
step = int(1.0/dt)
for t in range(step+1):
    rho_L = U[:, 0]
    u_L = U[:, 1] / rho_L
    v_L = U[:, 2] / rho_L
    w_L = U[:, 3] / rho_L
    Bx_L = U[:, 4]
    By_L = U[:, 5]
    Bz_L = U[:, 6]
    e_L = U[:, 7]
    rho_R = np.roll(rho_L, -1)
    u_R = np.roll(u_L[:, 1], -1)
    v_R = np.roll(v_L[:, 2], -1)
    w_R = np.roll(w_L[:, 3], -1)
    Bx_R = np.roll(Bx_L[:, 4], -1)
    By_R = np.roll(By_L[:, 5], -1)
    Bz_R = np.roll(Bz_L[:, 6], -1)
    e_R = np.roll(e_L[:, 7], -1)
    p_L = (gamma-1) * (e_L - rho_L*(u_L**2+v_L**2+w_L**2)/2 - (Bx_L**2+By_L**2+Bz_L**2)/2)
    p_R = np.roll(p_L, -1)

    cs_L = np.sqrt(gamma*p_L/rho_L)
    cs_R = np.roll(cs_L, -1)
    ca_L = np.sqrt(Bx_L**2+By_L**2+Bz_L**2) / np.sqrt(rho_L)
    ca_R = np.roll(ca_L, -1)
    fast_Alfven_L = 1/np.sqrt(2) * (cs_L**2 + ca_L**2 + np.sqrt((cs_L**2+ca_L**2)**2 - 4 * cs_L**2 * ca_L**2))
    fast_Alfven_R = np.roll(fast_Alfven_L, -1)
    S_L = np.minimum(u_L, u_R) - np.maximum(fast_Alfven_L, fast_Alfven_R)
    S_R = np.minimum(u_L, u_R) + np.maximum(fast_Alfven_L, fast_Alfven_R)

    pT_L = p_L + (Bx_L**2+By_L**2+Bz_L**2)/2
    pT_R = np.roll(pT_L, -1)

    SM = ((S_R - u_R)*rho_R*u_R - (S_L - u_L)*rho_L*u_L - pT_R + pT_L) / ((S_R - u_R)*rho_R - (S_L - u_L)*rho_L)
    pT1 = ((S_R - u_R)*rho_R*pT_L - (S_L - u_L)*rho_L*pT_R + rho_L*rho_R*(S_R - u_R)*(S_L - u_L)*(u_R - u_L)) / ((S_R - u_R)*rho_R - (S_L - u_L)*rho_L)
    rho1_L = rho_L * (S_L - u_L) / (S_L - SM)
    rho1_R = rho_R * (S_R - u_R) / (S_R - SM)
    u1_L, v1_L, w1_L, Bx1_L, By1_L, Bz1_L, e1_L = get_U_parameters_1(rho_L, u_L, v_L, w_L, Bx_L, By_L, Bz_L, e_L, pT1, S_L, SM)
    u1_R, v1_R, w1_R, Bx1_R, By1_R, Bz1_R, e1_R = get_U_parameters_1(rho_R, u_R, v_R, w_R, Bx_R, By_R, Bz_R, e_R, pT1, S_R, SM)

    S_L1 = SM - np.abs(Bx_L)/np.sqrt(rho1_L) #Bxの扱いはどうする？　1次元だからこれでいいはず…
    S_R1 = SM + np.abs(Bx_L)/np.sqrt(rho1_R) #Bxの扱いはどうする？　1次元だからこれでいいはず…

    u2, v2, w2, Bx2, By2, Bz2, e2_L, e2_R = get_U_parameters_2(rho1_L, rho1_R, u1_L, u1_R, v1_L, v1_R, w1_L, w1_R, Bx1_L, Bx1_R, By1_L, By1_R, Bz1_L, Bz1_R, e1_L, e1_R, SM)

    # Fの選択

    # 時間発展
    

TypeError: minimum() takes from 2 to 3 positional arguments but 0 were given